# PPO Training with GPU Acceleration

使用向量化环境进行批量 rollout，大幅提升 GPU 利用率和训练速度。
支持 A100 等高端 GPU 高效训练 ConnectX 智能体。

## 克隆代码仓库（云端首次运行需要）

In [ ]:
!git clone https://github.com/mogoo7zn/Kaggle-ConnectX.git
%cd Kaggle-ConnectX

## 0. 环境与依赖
- 需 GPU 环境（Kaggle GPU / Colab Pro / 本地 GPU）
- 安装最小依赖：`kaggle-environments`（对战模拟）、`huggingface_hub`（可选上传）

In [ ]:
!pip install -q kaggle-environments huggingface_hub

In [ ]:
import os
import sys
import random
import numpy as np
import torch
import torch.nn.functional as F
from pathlib import Path
import importlib
import time
from typing import Dict, Any

# 让项目包可被导入：优先常见 Kaggle 路径，否则尝试当前/父目录
candidates = [
    Path('/kaggle/working/Kaggle-ConnectX'),  # git clone 默认位置
    Path('/kaggle/working'),                  # 若 notebook 已位于仓库内，这里无 agents 则会跳过
    Path.cwd(),
    Path.cwd().parent,
]
repo_root = None
for c in candidates:
    if (c / 'agents').exists():
        repo_root = c
        break
if repo_root is None:
    repo_root = Path('.').resolve()

os.chdir(repo_root)
sys.path.insert(0, str(repo_root))
importlib.invalidate_caches()
print("Repo root:", repo_root)
print("CWD:", Path.cwd())
print("agents exists:", (repo_root / 'agents').exists())

from agents.ppo.ppo_agent import PPOAgent
from agents.ppo.ppo_config import ppo_config
from agents.base.utils import get_valid_moves, get_negamax_move, encode_state, make_move, is_terminal
from agents.dqn.dqn_agent import DQNAgent

# 可选：AlphaZero 载入（需要与你队友的权重匹配）
try:
    from agents.alphazero.az_model import create_alphazero_model
except Exception:
    import traceback
    traceback.print_exc()
    create_alphazero_model = None

DEVICE = ppo_config.DEVICE
print("Using device:", DEVICE)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory // 1024**3} GB")

## 1. 模型架构优化
使用更大的网络容量以充分利用 GPU 计算能力

In [ ]:
# 可在 notebook 内直接定义/替换模型，避免每次修改后重新 clone
# 调整下方通道/隐藏层尺寸即可放大模型规模
import types
import torch.nn as nn
import torch.nn.functional as F
from agents.ppo import ppo_model

class ActorCriticGPU(nn.Module):
    """
    针对 GPU 优化的更大网络架构
    """
    def __init__(self):
        super().__init__()
        # 大幅增加通道数以充分利用 GPU
        self.conv1 = nn.Conv2d(3, 256, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(256)
        self.conv2 = nn.Conv2d(256, 512, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(512)
        self.conv3 = nn.Conv2d(512, 512, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(512)
        
        conv_out = ppo_config.ROWS * ppo_config.COLUMNS * 512
        self.fc1 = nn.Linear(conv_out, 2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.dropout = nn.Dropout(0.1)
        
        self.policy = nn.Linear(1024, ppo_config.COLUMNS)
        self.value = nn.Linear(1024, 1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        logits = self.policy(x)
        value = self.value(x)
        return logits, value

# 覆盖原有工厂方法
ppo_model.ActorCritic = ActorCriticGPU
ppo_model.make_model = lambda: ActorCriticGPU().to(ppo_config.DEVICE)

print("Using GPU-optimized ActorCritic with large capacity:", ActorCriticGPU())
print(f"Model parameters: {sum(p.numel() for p in ActorCriticGPU().parameters()):,}")

## 2. 下载队友的已训练模型
作为冻结对手加入训练池

In [ ]:
from pathlib import Path
import requests

# 直接填入公开仓库的 resolve 链接
DQN_URL = "https://huggingface.co/mogoo7zn/Kaggle-ConnectX/resolve/main/DQN-base.pth"
AZ_URL  = "https://huggingface.co/mogoo7zn/Kaggle-ConnectX/resolve/main/alpha-zero-medium.pth"
# 如果你想用 high 版，就把上面这一行改成 alpha-zero-high.pth

ckpt_dir = Path('/kaggle/working/checkpoints')
ckpt_dir.mkdir(parents=True, exist_ok=True)

def download(url, out_path):
    if not url:
        print(f"[skip] empty url for {out_path.name}")
        return None
    print(f"Downloading {url} -> {out_path}")
    r = requests.get(url)
    r.raise_for_status()
    with open(out_path, 'wb') as f:
        f.write(r.content)
    return out_path

ckpt_dqn = download(DQN_URL, ckpt_dir/'dqn_frozen.pth')
ckpt_az  = download(AZ_URL,  ckpt_dir/'alphazero_frozen.pth')

## 3. 加载冻结对手
DQN 和 AlphaZero 作为固定难度的对手

In [ ]:
frozen_dqn = None
if ckpt_dqn and ckpt_dqn.exists():
    frozen_dqn = DQNAgent()
    frozen_dqn.load_model(str(ckpt_dqn))
    # 确保冻结模型也在 GPU 上
    frozen_dqn.model.to(DEVICE)
    frozen_dqn.model.eval()
    print("Loaded frozen DQN on GPU.")
else:
    print("No DQN checkpoint provided.")

frozen_az = None
if ckpt_az and ckpt_az.exists() and create_alphazero_model:
    try:
        frozen_az = create_alphazero_model('full')  # 如权重是轻量版改为 'light'
        state = torch.load(ckpt_az, map_location=DEVICE)
        if isinstance(state, dict) and 'model_state_dict' in state:
            state = state['model_state_dict']
        frozen_az.load_state_dict(state, strict=False)
        frozen_az.to(DEVICE)
        frozen_az.eval()
        print("Loaded frozen AlphaZero on GPU.")
    except Exception as e:
        print("Failed to load AlphaZero model:", e)
        frozen_az = None
else:
    print("No AlphaZero checkpoint provided or loader unavailable.")

## 4. 定义对手池
包含多种难度层次的对手，重点训练搜索类对手

In [ ]:
from types import SimpleNamespace
from kaggle_environments.envs.connectx.connectx import negamax_agent

def random_policy(board, mark):
    moves = get_valid_moves(board)
    return random.choice(moves) if moves else 0

def negamax_simple_policy(board, mark, depth=4):
    # 原简化版：先手赢/堵 + 中心优先
    return get_negamax_move(board, mark, depth=depth)

def negamax_kaggle_policy(board, mark, depth=4):
    # kaggle-environments 自带 negamax 搜索；深度通过配置传入
    obs = SimpleNamespace(board=board, mark=mark)
    cfg = SimpleNamespace(rows=ppo_config.ROWS, columns=ppo_config.COLUMNS, inarow=ppo_config.INAROW,
                          timeout=1, actTimeout=1, depth=depth)
    return int(negamax_agent(obs, cfg))

def frozen_dqn_policy(board, mark):
    assert frozen_dqn is not None, "frozen_dqn not loaded"
    return frozen_dqn.select_action(board, mark, epsilon=0.0)

def frozen_alphazero_policy(board, mark):
    assert frozen_az is not None, "frozen_az not loaded"
    # AlphaZero uses MCTS normally; here greedily pick best policy head
    state = encode_state(board, mark)
    state_t = torch.from_numpy(state).float().unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        logits, _ = frozen_az(state_t)
        probs = F.softmax(logits, dim=-1)[0].cpu().numpy()
    valid = get_valid_moves(board)
    masked = np.full_like(probs, -1e9)
    masked[valid] = probs[valid]
    return int(masked.argmax())

# GPU 加速训练：大幅提高搜索对手的采样权重
opponent_candidates = [
    ("negamax_simple", negamax_simple_policy, 0.15),
    ("negamax_kaggle", negamax_kaggle_policy, 0.40),  # 重点训练搜索对手
    ("random", random_policy, 0.20),
]
if frozen_dqn:
    opponent_candidates.append(("frozen_dqn", frozen_dqn_policy, 0.125))
if frozen_az:
    opponent_candidates.append(("frozen_az", frozen_alphazero_policy, 0.125))

# 归一化权重，确保总和为 1
opponent_fns = [fn for _, fn, _ in opponent_candidates]
opponent_weights = np.array([w for _, _, w in opponent_candidates], dtype=float)
opponent_weights = opponent_weights / opponent_weights.sum()

opponent_pool = [name for name, _, _ in opponent_candidates]
print("Opponent pool:", opponent_pool)
print("Weights:", opponent_weights)
print(f"Total opponents: {len(opponent_pool)}")

## 5. GPU 加速训练配置
使用向量化环境进行批量 rollout，大幅提升 GPU 利用率

In [ ]:
agent = PPOAgent()

# GPU 加速配置
TOTAL_UPDATES = 500              # 总训练步数
ROLLOUT_STEPS = 2048             # 单次 rollout 步数（增加以提高 GPU 利用率）
USE_VECTORIZED = True           # 使用向量化环境
NUM_VECTORIZED_ENVS = 64        # 向量化环境数量（根据 GPU 内存调整）

# PPO 超参数优化
ppo_config.LR = 1e-4            # 学习率
ppo_config.PPO_EPOCHS = 4       # PPO 更新轮次
ppo_config.MINI_BATCHES = 8     # mini-batch 数量
ppo_config.CLIP_RANGE = 0.2     # PPO clipping 范围
ppo_config.VF_COEF = 0.5        # value loss 权重
ppo_config.ENT_COEF = 0.01      # entropy 权重
ppo_config.MAX_GRAD_NORM = 0.5  # 梯度裁剪

LOG_INTERVAL = 10
reward_log = []
performance_stats = {
    'gpu_utilization': [],
    'step_time': [],
    'memory_usage': []
}

print("GPU Accelerated PPO Training Configuration:")
print(f"Total Updates: {TOTAL_UPDATES}")
print(f"Rollout Steps: {ROLLOUT_STEPS}")
print(f"Vectorized Envs: {NUM_VECTORIZED_ENVS if USE_VECTORIZED else 'Disabled'}")
print(f"Learning Rate: {ppo_config.LR}")
print(f"Device: {ppo_config.DEVICE}")

# 预热 GPU
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    warmup_tensor = torch.randn(1000, 1000, device=DEVICE)
    del warmup_tensor
    torch.cuda.empty_cache()
    print("GPU warmed up")

## 6. GPU 加速训练循环
监控 GPU 利用率、内存使用和训练性能

In [ ]:
start_time = time.time()

for update in range(1, TOTAL_UPDATES + 1):
    step_start = time.time()
    
    # 采样对手
    opp_fn = random.choices(opponent_fns, weights=opponent_weights, k=1)[0]
    
    # GPU 加速 rollout
    if USE_VECTORIZED:
        batch = agent.generate_vectorized_rollout(opp_fn, ROLLOUT_STEPS, NUM_VECTORIZED_ENVS)
    else:
        batch = agent.generate_rollout(opp_fn, ROLLOUT_STEPS)
    
    # PPO 更新
    metrics = agent.update(batch)
    reward_log.append(batch.returns.mean().item())
    
    step_time = time.time() - step_start
    performance_stats['step_time'].append(step_time)
    
    # 监控 GPU 状态
    if torch.cuda.is_available():
        gpu_util = torch.cuda.utilization(DEVICE) if hasattr(torch.cuda, 'utilization') else 0
        memory_used = torch.cuda.memory_allocated(DEVICE) / 1024**3  # GB
        memory_total = torch.cuda.get_device_properties(DEVICE).total_memory / 1024**3
        performance_stats['gpu_utilization'].append(gpu_util)
        performance_stats['memory_usage'].append(memory_used)
    
    if update % LOG_INTERVAL == 0:
        avg_ret = float(np.mean(reward_log[-LOG_INTERVAL:]))
        avg_step_time = np.mean(performance_stats['step_time'][-LOG_INTERVAL:])
        
        # 安全地获取指标
        if isinstance(metrics, dict):
            loss_val = metrics.get("loss", metrics.get("total_loss", 0.0))
            policy = metrics.get("policy_loss", float("nan"))
            value = metrics.get("value_loss", float("nan"))
            ent = metrics.get("entropy", float("nan"))
            kl = metrics.get("approx_kl", float("nan"))
            clip = metrics.get("clip_frac", float("nan"))
        else:
            loss_val = float(metrics)
            policy = value = ent = kl = clip = float("nan")
        
        # GPU 性能指标
        gpu_info = ""
        if torch.cuda.is_available() and performance_stats['gpu_utilization']:
            avg_gpu = np.mean(performance_stats['gpu_utilization'][-LOG_INTERVAL:])
            avg_mem = np.mean(performance_stats['memory_usage'][-LOG_INTERVAL:])
            gpu_info = f" | GPU {avg_gpu:.1f}% | Mem {avg_mem:.1f}GB"
        
        elapsed = time.time() - start_time
        eta = (elapsed / update) * (TOTAL_UPDATES - update)
        
        print(
            f"Update {update}/{TOTAL_UPDATES} "
            f"| loss {loss_val:.3f} "
            f"| policy {policy:.3f} "
            f"| value {value:.3f} "
            f"| ent {ent:.3f} "
            f"| kl {kl:.4f} "
            f"| clip {clip:.3f} "
            f"| avg_ret {avg_ret:.3f} "
            f"| step {avg_step_time:.2f}s"
            f"{gpu_info} "
            f"| ETA {eta/3600:.1f}h"
        )

# 保存模型
ppo_path = Path('/kaggle/working/ppo_gpu_accelerated.pth')
torch.save(agent.model.state_dict(), ppo_path)
total_time = time.time() - start_time
print(f"\nTraining completed in {total_time/3600:.1f} hours")
print(f"Average step time: {np.mean(performance_stats['step_time']):.2f}s")
if performance_stats['gpu_utilization']:
    print(f"Average GPU utilization: {np.mean(performance_stats['gpu_utilization']):.1f}%")
print("Saved", ppo_path)

## 7. GPU 加速评估
使用向量化评估快速测试模型性能

In [ ]:
def play_one(policy_fn, opp_fn, games=50):
    """GPU 加速评估：增加对局数以获得更稳定的胜率估计"""
    wins = 0
    for g in range(games):
        board = [0]*(ppo_config.ROWS*ppo_config.COLUMNS)
        # 轮流先手：偶数局 PPO 先手(标记1)，奇数局对手先手(标记1)，PPO 用标记2
        ppo_first = (g % 2 == 0)
        ppo_mark = 1 if ppo_first else 2
        current = 1
        while True:
            if current == ppo_mark:
                action = policy_fn(board, ppo_mark)
            else:
                action = opp_fn(board, current)
            board = make_move(board, action, current)
            done, winner = is_terminal(board)
            if done:
                if winner == ppo_mark:
                    wins += 1
                break
            current = 3 - current
    return wins / games

def ppo_policy(board, mark):
    state = encode_state(board, mark)
    state_t = torch.from_numpy(state).float().unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        logits, _ = agent.model(state_t)
        valid = get_valid_moves(board)
        mask = torch.full_like(logits, float('-inf'))
        mask[0, valid] = 0
        logits = logits + mask
        probs = F.softmax(logits, dim=-1)[0].cpu().numpy()
    return int(np.random.choice(len(probs), p=probs))

# GPU 加速评估
print("GPU Accelerated Model Evaluation:")
print("="*50)

# 基础对手
print("PPO vs random:", play_one(ppo_policy, random_policy, games=50))

# 搜索对手
print("PPO vs negamax_simple:", play_one(ppo_policy, lambda b,m: negamax_simple_policy(b,m,depth=4), games=50))
print("PPO vs negamax_kaggle:", play_one(ppo_policy, lambda b,m: negamax_kaggle_policy(b,m,depth=4), games=50))

# 学习对手
if frozen_dqn:
    print("PPO vs frozen DQN:", play_one(ppo_policy, frozen_dqn_policy, games=50))
if frozen_az:
    print("PPO vs frozen AlphaZero:", play_one(ppo_policy, frozen_alphazero_policy, games=50))

print("="*50)
print("Evaluation completed with GPU acceleration!")

## 8. 性能分析
分析训练过程中的 GPU 利用率和性能指标

In [ ]:
import matplotlib.pyplot as plt

# 训练曲线
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 奖励曲线
axes[0,0].plot(reward_log, alpha=0.7)
axes[0,0].set_title('Training Reward')
axes[0,0].set_xlabel('Update')
axes[0,0].set_ylabel('Average Return')
axes[0,0].grid(True)

# GPU 利用率
if performance_stats['gpu_utilization']:
    gpu_log = performance_stats['gpu_utilization']
    axes[0,1].plot(gpu_log, alpha=0.7, color='green')
    axes[0,1].set_title('GPU Utilization')
    axes[0,1].set_xlabel('Update')
    axes[0,1].set_ylabel('GPU %')
    axes[0,1].set_ylim(0, 100)
    axes[0,1].grid(True)
    axes[0,1].axhline(y=50, color='red', linestyle='--', alpha=0.7, label='50% threshold')
    axes[0,1].legend()

# 内存使用
if performance_stats['memory_usage']:
    mem_log = performance_stats['memory_usage']
    axes[1,0].plot(mem_log, alpha=0.7, color='orange')
    axes[1,0].set_title('GPU Memory Usage')
    axes[1,0].set_xlabel('Update')
    axes[1,0].set_ylabel('Memory (GB)')
    axes[1,0].grid(True)

# 步长时间
step_times = performance_stats['step_time']
axes[1,1].plot(step_times, alpha=0.7, color='purple')
axes[1,1].set_title('Step Time')
axes[1,1].set_xlabel('Update')
axes[1,1].set_ylabel('Time (s)')
axes[1,1].grid(True)

plt.tight_layout()
plt.show()

# 性能统计
print("\nGPU Accelerated Training Statistics:")
print("="*50)
print(f"Total training time: {total_time/3600:.2f} hours")
print(f"Average step time: {np.mean(step_times):.3f}s")
print(f"Total steps: {len(step_times)}")
print(f"Total environment interactions: {len(step_times) * ROLLOUT_STEPS * (NUM_VECTORIZED_ENVS if USE_VECTORIZED else 1):,}")

if performance_stats['gpu_utilization']:
    print(f"Average GPU utilization: {np.mean(gpu_log):.1f}%")
    print(f"Peak GPU utilization: {np.max(gpu_log):.1f}%")

if performance_stats['memory_usage']:
    print(f"Average GPU memory: {np.mean(mem_log):.2f} GB")
    print(f"Peak GPU memory: {np.max(mem_log):.2f} GB")

print(f"Final average reward: {np.mean(reward_log[-50:]):.3f}")
print("="*50)

## 9. 上传到 Hugging Face
保存训练好的 GPU 加速模型到 HF

In [ ]:
import os
from huggingface_hub import HfApi, HfFolder

# 设置你的 HF token（安全方式：使用环境变量）
# os.environ["HUGGINGFACE_HUB_TOKEN"] = "your_token_here"

HF_TOKEN = os.environ.get("HUGGINGFACE_HUB_TOKEN", "")
REPO_ID = "your-username/connectx-ppo-gpu-accelerated"  # 替换为你的仓库

if HF_TOKEN and REPO_ID:
    api = HfApi()
    HfFolder.save_token(HF_TOKEN)
    
    # 创建或获取仓库
    try:
        api.create_repo(repo_id=REPO_ID, repo_type="model", private=False)
        print(f"Created HF repo: {REPO_ID}")
    except:
        print(f"HF repo {REPO_ID} already exists")
    
    # 上传模型
    api.upload_file(
        path_or_fileobj=str(ppo_path),
        path_in_repo="ppo_gpu_accelerated.pth",
        repo_id=REPO_ID,
        repo_type="model",
    )
    print(f"Uploaded GPU-accelerated model to HF: {REPO_ID}")
    
    # 创建模型卡片
    model_card = f"""
---
tags:
- connectx
- reinforcement-learning
- ppo
- gpu-accelerated
library_name: pytorch
---

# ConnectX PPO Agent (GPU Accelerated)

This model was trained using GPU-accelerated PPO with vectorized environments for maximum training efficiency.

## Model Details

- **Algorithm**: Proximal Policy Optimization (PPO)
- **Environment**: ConnectX (6x7 grid, 4-in-a-row)
- **Training**: GPU-accelerated with {NUM_VECTORIZED_ENVS} parallel environments
- **Architecture**: Large CNN (256→512→512 channels) + FC layers
- **Parameters**: {sum(p.numel() for p in ActorCriticGPU().parameters()):,}

## Training Configuration

- Total Updates: {TOTAL_UPDATES}
- Rollout Steps: {ROLLOUT_STEPS}
- Learning Rate: {ppo_config.LR}
- Vectorized Environments: {NUM_VECTORIZED_ENVS}
- GPU Memory: ~{np.max(performance_stats['memory_usage']):.1f} GB peak
- Training Time: {total_time/3600:.1f} hours

## Performance

- Average GPU Utilization: {np.mean(performance_stats['gpu_utilization']):.1f}%
- Final Average Reward: {np.mean(reward_log[-50:]):.3f}

## Usage

```python
import torch
from agents.ppo.ppo_model import make_model

model = make_model()
model.load_state_dict(torch.load('ppo_gpu_accelerated.pth'))
model.eval()
```
"""
    
    # 上传模型卡片
    api.upload_file(
        path_or_fileobj=model_card,
        path_in_repo="README.md",
        repo_id=REPO_ID,
        repo_type="model",
    )
    print("Uploaded model card")
    
else:
    print("HF token or repo id not set; skip upload.")
    print("To upload, set HUGGINGFACE_HUB_TOKEN environment variable and update REPO_ID")